# ReAct Framework: Reasoning and Acting

This notebook demonstrates the **ReAct (Reasoning and Acting) framework**, which synergistically combines reasoning traces and task-specific actions in an interleaved manner. ReAct enables AI agents to generate verbal reasoning traces while simultaneously taking actions, creating a powerful paradigm for autonomous problem-solving.

## Table of Contents
1. [Introduction to ReAct](#introduction)
2. [Environment Setup](#setup)
3. [Basic ReAct Pattern](#basic)
4. [ReAct with Tool Usage](#tools)
5. [Multi-Step ReAct Reasoning](#multi-step)
6. [Error Handling and Recovery](#error-handling)
7. [Advanced ReAct Patterns](#advanced)
8. [Summary and Best Practices](#summary)

## 1. Introduction to ReAct <a id="introduction"></a>

### What is ReAct?

**ReAct** (Reasoning and Acting) is a framework that combines:
- **Reasoning**: Generating thought traces that explain decision-making
- **Acting**: Taking actions in the environment using available tools

The key innovation is the interleaving of reasoning and action, creating a loop:
```
Thought → Action → Observation → Thought → Action → ...
```

### ReAct vs. Traditional Approaches

| Approach | Reasoning | Actions | Interleaved |
|----------|-----------|---------|-------------|
| Chain-of-Thought | ✓ | ✗ | ✗ |
| Action-Only Agents | ✗ | ✓ | ✗ |
| **ReAct** | **✓** | **✓** | **✓** |

### Benefits of ReAct

1. **Interpretability**: Reasoning traces explain why actions are taken
2. **Diagnosability**: Easy to identify where reasoning or execution fails
3. **Controllability**: Can guide behavior by modifying reasoning prompts
4. **Flexibility**: Adapts to new tools and environments easily
5. **Human-like**: Mirrors how humans think and act on problems

### Core Components

1. **Thought**: Internal reasoning about what to do next
2. **Action**: Specific tool or function call with parameters
3. **Observation**: Result returned from the action
4. **Decision**: Whether to continue or conclude

### Example ReAct Trace

```
Question: What is the capital of the country where the 2024 Olympics were held?

Thought: I need to first find out which country hosted the 2024 Olympics.
Action: search("2024 Olympics host country")
Observation: The 2024 Summer Olympics were held in Paris, France.

Thought: Now I know France hosted the Olympics. The capital of France is Paris.
Action: finish("Paris")
```

## 2. Environment Setup <a id="setup"></a>

Let's set up our environment and create some simple tools for the agent to use.

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
import os
import json
import re
from typing import List, Dict, Any, Tuple, Optional

# Load environment variables
load_dotenv()

# Initialize the AI Project client
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential()
)

# Get the OpenAI client
chat = project_client.get_openai_client()
model = os.environ["MODEL"]

print(f"Connected to Azure AI Foundry")
print(f"Using model: {model}")

In [ ]:
# Create a simple knowledge base for our tools to access
knowledge_base = {
    "azure_ai_foundry": {
        "description": "Azure AI Foundry is a platform for building, deploying, and managing AI applications",
        "features": ["model deployment", "prompt engineering", "evaluation", "monitoring"],
        "release_year": 2023
    },
    "rag": {
        "description": "Retrieval-Augmented Generation combines retrieval with text generation",
        "benefits": ["grounded responses", "reduced hallucination", "up-to-date information"],
        "components": ["retriever", "generator", "knowledge base"]
    },
    "react_framework": {
        "description": "ReAct combines reasoning and acting in an interleaved manner",
        "paper_year": 2022,
        "key_innovation": "synergistic combination of reasoning traces and actions"
    },
    "chain_of_thought": {
        "description": "Chain-of-Thought prompting encourages step-by-step reasoning",
        "invented_by": "Google Research",
        "paper_year": 2022
    },
    "microsoft_agent_framework": {
        "description": "Framework for building autonomous AI agents",
        "features": ["tool usage", "planning", "memory", "multi-agent orchestration"]
    }
}

print(f"Knowledge base loaded with {len(knowledge_base)} entries")

In [ ]:
# Define simple tools for the agent

def search_knowledge(query: str) -> str:
    """
    Search the knowledge base for information.
    Returns: Information about the topic or 'Not found'
    """
    query_lower = query.lower()
    
    # Simple keyword matching
    for key, value in knowledge_base.items():
        if key.replace('_', ' ') in query_lower or any(word in query_lower for word in key.split('_')):
            return json.dumps(value, indent=2)
    
    return "Not found. Available topics: " + ", ".join([k.replace('_', ' ') for k in knowledge_base.keys()])

def calculate(expression: str) -> str:
    """
    Safely evaluate mathematical expressions.
    Returns: Result of calculation or error message
    """
    try:
        # Only allow basic arithmetic for safety
        allowed_chars = set('0123456789+-*/()., ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic arithmetic operations allowed (+, -, *, /, ())"
        
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

def get_year_difference(year1: int, year2: int) -> str:
    """
    Calculate the difference between two years.
    Returns: Difference in years
    """
    try:
        diff = abs(int(year1) - int(year2))
        return f"{diff} years"
    except Exception as e:
        return f"Error: {str(e)}"

def finish(answer: str) -> str:
    """
    Provide the final answer.
    Returns: The final answer
    """
    return f"FINAL_ANSWER: {answer}"

# Tool registry
TOOLS = {
    "search_knowledge": {
        "function": search_knowledge,
        "description": "Search the knowledge base for information about AI topics. Input: query string."
    },
    "calculate": {
        "function": calculate,
        "description": "Perform mathematical calculations. Input: arithmetic expression as string."
    },
    "get_year_difference": {
        "function": get_year_difference,
        "description": "Calculate difference between two years. Input: year1, year2."
    },
    "finish": {
        "function": finish,
        "description": "Provide the final answer when you have enough information. Input: answer string."
    }
}

def get_tool_descriptions() -> str:
    """Get formatted descriptions of available tools"""
    descriptions = []
    for tool_name, tool_info in TOOLS.items():
        descriptions.append(f"{tool_name}: {tool_info['description']}")
    return "\n".join(descriptions)

print("Tools available:")
print(get_tool_descriptions())

## 3. Basic ReAct Pattern <a id="basic"></a>

Let's implement the basic ReAct loop: Thought → Action → Observation → repeat.

### 3.1 Simple ReAct Agent

In [ ]:
def parse_action(text: str) -> Optional[Tuple[str, str]]:
    """
    Parse action from model output.
    Expected format: Action: tool_name("argument")
    Returns: (tool_name, argument) or None
    """
    # Look for Action: pattern
    action_pattern = r'Action:\s*(\w+)\s*\(([^)]+)\)'
    match = re.search(action_pattern, text, re.IGNORECASE)
    
    if match:
        tool_name = match.group(1)
        argument = match.group(2).strip('"\' ')
        return (tool_name, argument)
    
    return None

def execute_action(tool_name: str, argument: str) -> str:
    """
    Execute a tool with given argument.
    Returns: Observation from the tool
    """
    if tool_name not in TOOLS:
        return f"Error: Unknown tool '{tool_name}'. Available tools: {', '.join(TOOLS.keys())}"
    
    try:
        # Handle tools with multiple arguments
        if tool_name == "get_year_difference":
            args = [arg.strip() for arg in argument.split(',')]
            return TOOLS[tool_name]["function"](*args)
        else:
            return TOOLS[tool_name]["function"](argument)
    except Exception as e:
        return f"Error executing {tool_name}: {str(e)}"

def simple_react_agent(question: str, max_iterations: int = 5, verbose: bool = True) -> Dict[str, Any]:
    """
    Simple ReAct agent that reasons and acts.
    """
    # Build the ReAct prompt
    react_prompt = f"""Answer the following question using this format:

Question: [the question]
Thought: [your reasoning about what to do]
Action: tool_name("argument")
Observation: [result from the action]
... (repeat Thought/Action/Observation as needed)
Thought: [final reasoning]
Action: finish("final answer")

Available tools:
{get_tool_descriptions()}

Question: {question}
"""
    
    conversation = react_prompt
    trace = []
    
    if verbose:
        print("=" * 80)
        print(f"Question: {question}")
        print("=" * 80)
        print()
    
    for iteration in range(max_iterations):
        # Get model's thought and action
        response = chat.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": conversation}],
            temperature=0.3,
            max_tokens=300
        )
        
        response_text = response.choices[0].message.content
        
        if verbose:
            print(response_text)
            print()
        
        trace.append({"iteration": iteration + 1, "response": response_text})
        
        # Check if finished
        if "FINAL_ANSWER:" in response_text:
            # Extract final answer
            final_answer = response_text.split("FINAL_ANSWER:")[1].strip()
            return {
                "answer": final_answer,
                "iterations": iteration + 1,
                "trace": trace,
                "status": "success"
            }
        
        # Parse and execute action
        action_info = parse_action(response_text)
        
        if action_info:
            tool_name, argument = action_info
            observation = execute_action(tool_name, argument)
            
            if verbose:
                print(f"Observation: {observation}")
                print("-" * 80)
                print()
            
            # Add observation to conversation
            conversation += f"\n{response_text}\nObservation: {observation}\n"
            trace[-1]["action"] = {"tool": tool_name, "argument": argument}
            trace[-1]["observation"] = observation
            
            # Check if this was the finish action
            if tool_name == "finish":
                return {
                    "answer": observation.replace("FINAL_ANSWER: ", ""),
                    "iterations": iteration + 1,
                    "trace": trace,
                    "status": "success"
                }
        else:
            if verbose:
                print("Warning: No valid action found in response")
            conversation += f"\n{response_text}\n"
    
    return {
        "answer": "Max iterations reached without finding answer",
        "iterations": max_iterations,
        "trace": trace,
        "status": "max_iterations"
    }

# Test the simple ReAct agent
result = simple_react_agent("What is RAG and what are its benefits?")

print("=" * 80)
print(f"Final Answer: {result['answer']}")
print(f"Completed in {result['iterations']} iterations")
print("=" * 80)

### 3.2 ReAct with Multi-Step Reasoning

In [ ]:
# Test with a question requiring multiple steps
multi_step_question = """How many years after Chain-of-Thought was invented did the ReAct framework paper come out?"""

print("Multi-Step ReAct Example:")
print("=" * 80)
result = simple_react_agent(multi_step_question)

print("=" * 80)
print(f"Final Answer: {result['answer']}")
print(f"Steps taken: {result['iterations']}")
print("=" * 80)

## 4. ReAct with Tool Usage <a id="tools"></a>

### 4.1 Enhanced ReAct with Better Tool Handling

In [ ]:
def enhanced_react_agent(question: str, max_iterations: int = 7, verbose: bool = True) -> Dict[str, Any]:
    """
    Enhanced ReAct agent with better prompting and error handling.
    """
    system_prompt = """You are an AI assistant that uses the ReAct (Reasoning and Acting) framework.

For each step:
1. Think about what information you need
2. Choose and execute an appropriate action
3. Observe the result
4. Repeat until you can provide a final answer

Always use this exact format:
Thought: [your reasoning]
Action: tool_name("argument")

After receiving an observation, continue with another Thought/Action cycle or provide the final answer using finish().
"""
    
    user_prompt = f"""Available tools:
{get_tool_descriptions()}

Question: {question}

Let's solve this step by step using the ReAct framework."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    trace = []
    
    if verbose:
        print("=" * 80)
        print(f"Question: {question}")
        print("=" * 80)
        print()
    
    for iteration in range(max_iterations):
        # Get model response
        response = chat.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.3,
            max_tokens=400
        )
        
        response_text = response.choices[0].message.content
        messages.append({"role": "assistant", "content": response_text})
        
        if verbose:
            print(f"Step {iteration + 1}:")
            print("-" * 80)
            print(response_text)
            print()
        
        trace.append({"iteration": iteration + 1, "response": response_text})
        
        # Parse action
        action_info = parse_action(response_text)
        
        if action_info:
            tool_name, argument = action_info
            observation = execute_action(tool_name, argument)
            
            trace[-1]["action"] = {"tool": tool_name, "argument": argument}
            trace[-1]["observation"] = observation
            
            if verbose:
                print(f"Observation: {observation}")
                print("=" * 80)
                print()
            
            # Check for completion
            if "FINAL_ANSWER:" in observation:
                final_answer = observation.replace("FINAL_ANSWER: ", "").strip()
                return {
                    "answer": final_answer,
                    "iterations": iteration + 1,
                    "trace": trace,
                    "status": "success"
                }
            
            # Add observation to conversation
            messages.append({"role": "user", "content": f"Observation: {observation}"})
        else:
            # No valid action found, prompt for one
            if verbose:
                print("No action found, prompting for action...")
                print()
            messages.append({"role": "user", "content": "Please provide your next Thought and Action."})
    
    return {
        "answer": "Max iterations reached",
        "iterations": max_iterations,
        "trace": trace,
        "status": "max_iterations"
    }

# Test enhanced agent
question = "What are the features of Azure AI Foundry and how many are there?"
result = enhanced_react_agent(question)

print("=" * 80)
print("FINAL RESULT")
print("=" * 80)
print(f"Answer: {result['answer']}")
print(f"Status: {result['status']}")
print(f"Iterations: {result['iterations']}")
print("=" * 80)

## 5. Multi-Step ReAct Reasoning <a id="multi-step"></a>

### 5.1 Complex Multi-Step Problem

In [ ]:
# Add more knowledge for a complex question
knowledge_base["gpt4"] = {
    "description": "GPT-4 is a large multimodal model from OpenAI",
    "release_year": 2023,
    "capabilities": ["text generation", "image understanding", "code generation"]
}

complex_question = """If Chain-of-Thought was invented in 2022 and GPT-4 was released in 2023,
how many years passed between them? Also, what are GPT-4's capabilities?"""

print("Complex Multi-Step ReAct Example:")
result = enhanced_react_agent(complex_question, max_iterations=10)

print("=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Question: {complex_question}")
print(f"\nAnswer: {result['answer']}")
print(f"\nSteps taken: {result['iterations']}")
print(f"Status: {result['status']}")

# Show the reasoning trace
print("\n" + "=" * 80)
print("REASONING TRACE")
print("=" * 80)
for step in result['trace']:
    print(f"\nStep {step['iteration']}:")
    if 'action' in step:
        print(f"  Action: {step['action']['tool']}({step['action']['argument']})")
        print(f"  Result: {step['observation'][:100]}..." if len(step['observation']) > 100 else f"  Result: {step['observation']}")

## 6. Error Handling and Recovery <a id="error-handling"></a>

### 6.1 ReAct with Error Recovery

In [ ]:
def react_with_recovery(question: str, max_iterations: int = 8, verbose: bool = True) -> Dict[str, Any]:
    """
    ReAct agent that can recover from errors.
    """
    system_prompt = """You are an AI assistant using the ReAct framework.

When you encounter an error:
1. Acknowledge what went wrong
2. Adjust your approach
3. Try a different action or argument

Use this format:
Thought: [reasoning]
Action: tool_name("argument")

If you get an error, think about why and try again with a corrected approach."""
    
    user_prompt = f"""Available tools:
{get_tool_descriptions()}

Question: {question}

Solve this using ReAct. If you encounter errors, learn from them and adjust."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    trace = []
    errors_encountered = 0
    
    if verbose:
        print("=" * 80)
        print(f"Question: {question}")
        print("=" * 80)
        print()
    
    for iteration in range(max_iterations):
        response = chat.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.4,
            max_tokens=400
        )
        
        response_text = response.choices[0].message.content
        messages.append({"role": "assistant", "content": response_text})
        
        if verbose:
            print(f"Step {iteration + 1}:")
            print(response_text)
            print()
        
        trace.append({"iteration": iteration + 1, "response": response_text})
        
        action_info = parse_action(response_text)
        
        if action_info:
            tool_name, argument = action_info
            observation = execute_action(tool_name, argument)
            
            # Check for errors
            if "Error" in observation:
                errors_encountered += 1
                if verbose:
                    print(f"❌ Error encountered: {observation}")
                    print()
            
            trace[-1]["action"] = {"tool": tool_name, "argument": argument}
            trace[-1]["observation"] = observation
            trace[-1]["is_error"] = "Error" in observation
            
            if verbose and "Error" not in observation:
                print(f"✓ Observation: {observation}")
                print("-" * 80)
                print()
            
            if "FINAL_ANSWER:" in observation:
                final_answer = observation.replace("FINAL_ANSWER: ", "").strip()
                return {
                    "answer": final_answer,
                    "iterations": iteration + 1,
                    "errors_encountered": errors_encountered,
                    "trace": trace,
                    "status": "success"
                }
            
            messages.append({"role": "user", "content": f"Observation: {observation}"})
    
    return {
        "answer": "Max iterations reached",
        "iterations": max_iterations,
        "errors_encountered": errors_encountered,
        "trace": trace,
        "status": "max_iterations"
    }

# Test with a question that might cause errors
error_question = "What is 2024 minus the release year of Azure AI Foundry?"

result = react_with_recovery(error_question)

print("=" * 80)
print("RESULT")
print("=" * 80)
print(f"Answer: {result['answer']}")
print(f"Errors encountered and recovered from: {result['errors_encountered']}")
print(f"Total iterations: {result['iterations']}")
print("=" * 80)

## 7. Advanced ReAct Patterns <a id="advanced"></a>

### 7.1 ReAct with Self-Reflection

In [ ]:
def react_with_reflection(question: str, max_iterations: int = 10, verbose: bool = True) -> Dict[str, Any]:
    """
    ReAct agent that periodically reflects on its progress.
    """
    system_prompt = """You are an advanced AI assistant using the ReAct framework with self-reflection.

Every few steps, pause to reflect:
- Am I making progress toward the answer?
- Is my current approach working?
- Should I try a different strategy?

Use these action types:
- Regular actions: tool_name("argument")
- Reflection: After every 2-3 actions, add a "Reflection:" statement

Format:
Thought: [reasoning]
Action: tool_name("argument")
[After observation]
Reflection: [evaluate progress and plan next steps]"""
    
    user_prompt = f"""Available tools:
{get_tool_descriptions()}

Question: {question}

Solve this using ReAct with periodic self-reflection."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    trace = []
    actions_taken = 0
    
    if verbose:
        print("=" * 80)
        print(f"Question: {question}")
        print("=" * 80)
        print()
    
    for iteration in range(max_iterations):
        response = chat.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.4,
            max_tokens=500
        )
        
        response_text = response.choices[0].message.content
        messages.append({"role": "assistant", "content": response_text})
        
        if verbose:
            print(f"Step {iteration + 1}:")
            print("-" * 80)
            print(response_text)
            print()
        
        trace.append({
            "iteration": iteration + 1,
            "response": response_text,
            "has_reflection": "reflection:" in response_text.lower()
        })
        
        action_info = parse_action(response_text)
        
        if action_info:
            actions_taken += 1
            tool_name, argument = action_info
            observation = execute_action(tool_name, argument)
            
            trace[-1]["action"] = {"tool": tool_name, "argument": argument}
            trace[-1]["observation"] = observation
            
            if verbose:
                print(f"Observation: {observation}")
                print("=" * 80)
                print()
            
            if "FINAL_ANSWER:" in observation:
                final_answer = observation.replace("FINAL_ANSWER: ", "").strip()
                reflections = sum(1 for t in trace if t.get("has_reflection", False))
                return {
                    "answer": final_answer,
                    "iterations": iteration + 1,
                    "actions_taken": actions_taken,
                    "reflections": reflections,
                    "trace": trace,
                    "status": "success"
                }
            
            messages.append({"role": "user", "content": f"Observation: {observation}"})
            
            # Prompt for reflection every 2 actions
            if actions_taken % 2 == 0 and actions_taken > 0:
                messages.append({"role": "user", "content": "Please reflect on your progress before continuing."})
    
    return {
        "answer": "Max iterations reached",
        "iterations": max_iterations,
        "actions_taken": actions_taken,
        "trace": trace,
        "status": "max_iterations"
    }

# Test ReAct with reflection
reflection_question = """What is the key innovation of the ReAct framework, and how does it relate to 
the components of RAG systems?"""

result = react_with_reflection(reflection_question)

print("=" * 80)
print("SUMMARY")
print("=" * 80)
print(f"Answer: {result['answer']}")
print(f"Actions taken: {result.get('actions_taken', 'N/A')}")
print(f"Reflections: {result.get('reflections', 'N/A')}")
print(f"Total iterations: {result['iterations']}")
print("=" * 80)

## 8. Summary and Best Practices <a id="summary"></a>

### Key Takeaways

**ReAct Framework Essentials:**

1. **Core Loop**: Thought → Action → Observation → (repeat)
2. **Interleaving**: Reasoning and acting happen together, not separately
3. **Transparency**: Reasoning traces explain every decision
4. **Flexibility**: Easy to add new tools and capabilities
5. **Robustness**: Can recover from errors through reasoning

### Implementation Best Practices

**Prompt Design:**
```
✓ Clear format specification (Thought/Action/Observation)
✓ Explicit tool descriptions
✓ System message defining the ReAct pattern
✓ Examples for complex scenarios
✗ Vague instructions
✗ Mixing action formats
```

**Action Parsing:**
- Use regex for consistent action extraction
- Handle variations in formatting gracefully
- Provide clear error messages for invalid actions
- Support both quoted and unquoted arguments

**Tool Design:**
- Keep tools focused and single-purpose
- Provide clear, informative descriptions
- Return structured, parseable results
- Include error handling in every tool
- Document expected input formats

**Error Handling:**
- Let the agent see and reason about errors
- Provide actionable error messages
- Allow the agent to retry with different approaches
- Set iteration limits to prevent infinite loops

**Reflection and Self-Correction:**
- Prompt for periodic reflection on progress
- Enable the agent to critique its own actions
- Allow strategy changes based on observations
- Track and learn from repeated errors

### Comparison with Other Patterns

| Pattern | Strengths | Weaknesses | Best For |
|---------|-----------|------------|----------|
| **ReAct** | Transparent, flexible, robust | More API calls, complex parsing | Multi-step tasks with tools |
| Chain-of-Thought | Simple, effective | No actions, less interactive | Pure reasoning tasks |
| Function Calling | Native support, reliable | Less transparent reasoning | Single-tool interactions |
| Agentic RAG | Knowledge-grounded | Requires good retrieval | Information-intensive tasks |

### Production Considerations

1. **Latency**: Each iteration adds API latency
   - Solution: Parallel tool execution when possible
   - Cache frequently used tool results

2. **Cost**: Multiple iterations increase token usage
   - Solution: Set reasonable iteration limits
   - Use smaller models for simple steps

3. **Reliability**: Parsing can fail on unexpected formats
   - Solution: Robust parsing with fallbacks
   - Retry with clearer instructions

4. **Monitoring**: Track agent behavior and success rates
   - Log all reasoning traces
   - Monitor tool usage patterns
   - Measure task completion rates

5. **Safety**: Agents can take unexpected actions
   - Solution: Validate tool inputs
   - Implement tool whitelisting
   - Add human approval for sensitive actions

### Advanced Techniques

1. **Multi-Agent ReAct**: Multiple ReAct agents collaborating
2. **Hierarchical ReAct**: Decompose into sub-tasks, each with ReAct
3. **ReAct + RAG**: Combine with retrieval for knowledge-grounded actions
4. **ReAct + Planning**: Add explicit planning phase before acting
5. **Meta-ReAct**: Agent reasons about which reasoning strategy to use

### When to Use ReAct

**Good Use Cases:**
- Tasks requiring multiple tool calls
- Problems needing exploratory approaches
- Scenarios where transparency is important
- Complex workflows with decision points
- Error-prone environments needing recovery

**Maybe Not Ideal:**
- Simple single-step questions
- Ultra-low latency requirements
- Pure generation tasks without tools
- Highly cost-sensitive applications

### Integration with Azure AI Foundry

ReAct works well with:
- **Azure AI Search**: As a retrieval tool
- **Function Calling**: For structured tool usage
- **Agent Framework**: As the reasoning engine
- **Evaluation Tools**: For measuring agent performance

### Next Steps

- Implement ReAct with real-world tools (APIs, databases, etc.)
- Combine ReAct with other techniques (RAG, planning)
- Build evaluation harnesses for agent quality
- Explore Microsoft Agent Framework integration
- Create domain-specific ReAct agents

### Additional Resources

- [ReAct Paper](https://arxiv.org/abs/2210.03629) - Original research
- [Microsoft Agent Framework](https://learn.microsoft.com/azure/ai-studio/)
- [LangChain ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react)
- [Azure AI Foundry Agents](https://learn.microsoft.com/azure/ai-studio/concepts/agents)

## Practice Exercises

Try building these enhancements:

1. **Add New Tools**: Implement additional tools (web search, database query, etc.)
2. **Tool Composition**: Enable using output from one tool as input to another
3. **Parallel Actions**: Execute multiple independent actions simultaneously
4. **Memory**: Add persistent memory across ReAct sessions
5. **Planning**: Add an explicit planning phase before ReAct execution
6. **Multi-Agent**: Create multiple ReAct agents that collaborate

In [ ]:
# Your practice code here
# Try implementing one of the exercises above!